#**INTRODUZIONE**

Nel nostro progetto, abbiamo deciso di utilizzare un modello **encoder** come `BERT-base-cased`, per sfruttare il suo potente meccanismo di rappresentazione contestuale del linguaggio.

Basandoci sul funzionamento di tale modello e sui dataset, abbiamo deciso di effettuare del pre-processing per cercare di massimizzare le informazioni estrapolabili dal testo, cercando di non danneggiarne il contenuto. Abbiamo dato molta importanza alle emoji e alle emoticon, le quali sono state aggiunte al **tokenizer** del modello.

Dato che nel dataset di train i campioni delle quattro classi erano fortemente sbilanciate abbiamo deciso di applicare la data augmentation.

Per adattare `BERT-base-cased` al linguaggio dei tweet, abbiamo eseguito un pre-training leggero con **Masked Language Modeling** (**MLM**), in modo da migliorare la rappresentazione dei nuovi token senza alterare significativamente il modello pre-addestrato.

Abbiamo infine effettuato il **fine-tuning** per adattare l'architettura alla classificazione, utilizzando una rete ricorrente `LSTM` per catturare le dipendenze sequenziali tra i token e migliorare la rappresentazione contestuale dei tweet.

Il nostro risultato è di un **F1-score** pari a 0.7589 sul dataset di *validation*, quasi identico al punteggio sul dataset di *test*.<br>
Abbiamo valutato l'uso di `BERTweet`, ipotizzando un netto miglioramento grazie al suo pre-training specifico su dati di Twitter. Nonostante un miglioramento (minimo) di 0.05 punti nell'**F1-score** con `BERTweet`, abbiamo preferito mantenere `BERT-base-cased`, in quanto già analizzato e ottimizzato durante la fase di progettazione.

#**IMPORT E CARICAMENTO DEL MODELLO**

In [ ]:
!pip install emot
import pandas as pd
import numpy as np
import re
import string
import nltk
import html
import emot
import sklearn
import json
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.optim import Adam, AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, BertModel, get_scheduler
from transformers import BertForMaskedLM, DataCollatorForLanguageModeling
config = AutoConfig.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model_lm = BertForMaskedLM.from_pretrained('bert-base-cased')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**CREAZIONE DEI DATAFRAME**

**CREZAZIONE DEL DATAFRAME DI TRAIN**

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion"
with open (path + "/train_text.txt", "r") as f:
    row_text = f.readlines()
# inserimento delle righe lette dal file all'interno di un DataFrame tramite Pandas
df_train = pd.DataFrame(row_text, columns = ["text"])


with open(path +"/train_labels.txt", "r") as f:
    row_labels = f.readlines()
# eliminazione di "\n" alla fine di ogni label e aggiunta al dataframe esistente
df_train["labels"] = [label.strip() for label in row_labels]

print(df_train)

                                                   text labels
0     “Worry is a down payment on a problem you may ...      2
1     My roommate: it's okay that we can't spell bec...      0
2     No but that's so cute. Atsu was probably shy a...      1
3     Rooneys fucking untouchable isn't he? Been fuc...      0
4     it's pretty depressing when u hit pan on ur fa...      3
...                                                 ...    ...
3252  I get discouraged because I try for 5 fucking ...      3
3253  The @user are in contention and hosting @user ...      3
3254  @user @user @user @user @user as a fellow UP g...      0
3255  You have a #problem? Yes! Can you do #somethin...      0
3256  @user @user i will fight this guy! Don't insul...      0

[3257 rows x 2 columns]


**CREAZIONE DEL DATAFRAME DI VALIDATION**

In [ ]:
with open (path + "/val_text.txt", "r") as f:
    row_text = f.readlines()
# inserimento delle righe lette dal file all'interno di un DataFrame tramite Pandas
df_val = pd.DataFrame(row_text, columns = ["text"])


with open(path + "/val_labels.txt", "r") as f:
    row_labels = f.readlines()
# eliminazione di "\n" alla fine di ogni label e aggiunta al dataframe esistente
df_val["labels"] = [label.strip() for label in row_labels]

print(df_val)

                                                  text labels
0    @user @user Oh, hidden revenge and anger...I r...      0
1    if not then #teamchristine bc all tana has don...      0
2    Hey @user #Fields in #skibbereen give your onl...      0
3    Why have #Emmerdale had to rob #robron of havi...      0
4    @user I would like to hear a podcast of you go...      0
..                                                 ...    ...
369  @user @user If #trump #whitehouse aren't held ...      0
370  @user Which #chutiya #producer #invested in #c...      0
371  Russia story will infuriate Trump today. Media...      0
372                      Shit getting me irritated 😠        0
373  @user @user If this didn't make me so angry, I...      0

[374 rows x 2 columns]


**CREAZIONE DEL DATAFRAME DI TEST**

In [ ]:
with open (path +"/test_text.txt", "r") as f:
    row_text = f.readlines()

# inserimento delle righe lette dal file all'interno di un DataFrame tramite Pandas
df_test = pd.DataFrame(row_text, columns = ["text"])

print(df_test)

                                                   text
0     #Deppression is real. Partners w/ #depressed p...
1     @user Interesting choice of words... Are you c...
2     My visit to hospital for care triggered #traum...
3     @user Welcome to #MPSVT! We are delighted to h...
4                        What makes you feel #joyful?  
...                                                 ...
1416  I need a sparkling bodysuit . No occasion. Jus...
1417  @user I've finished reading it; simply mind-bl...
1418  shaft abrasions from panties merely shifted to...
1419      All this fake outrage. Y'all need to stop 🤣  
1420  Would be ever so grateful if you could record ...

[1421 rows x 1 columns]


**CREAZIONE DEL DIZIONARIO DI SLANG E ABBREVIAZIONI**

Abbiamo deciso di creare un dizionario che contiene gli slang e le abbreviazioni più utilizzate nel web.<br>
Verrà utilizzato per il pre-processing di tutti e tre i dataset in modo da normalizzare queste forme e tradurle in una forma testuale estesa.

In [ ]:
# il file "abbreviations.txt" è stato creato da noi con le abbreviazioni e gli slang più usati
slang_dict = {}
with open (path + "/abbreviations.txt", "r") as f:
    for line in f:
        key, value = line.strip().split(": ")
        slang_dict[key.strip()] = value.strip()

print(slang_dict)

{'af': 'very', 'afaik': 'as far as i know', 'afair': 'as far as i remember', 'afaic': "as far as i'm concerned", 'afk': 'away from keyboard', 'asap': 'as soon as possible', 'atk': 'at the keyboard', 'atm': 'at the moment', 'bae': 'baby', 'bbl': 'be back later', 'bbs': 'be back soon', 'bff': 'best friends forever', 'bfn': 'bye for now', 'bloody': 'very', 'b4n': 'by for now', 'brb': 'be right back', 'brt': 'be right there', 'btw': 'by the way', 'b4': 'before', 'cheeky': 'playful', 'chill': 'relax', 'csl': "can't stop laughing", 'cu': 'see you', 'cul8r': 'see you later', 'cya': 'see you', 'dae': 'does anyone else?', 'dm': 'direct message', 'dodgy': 'suspicious', 'dope': 'cool', 'dude': 'guy', 'dunno': "don't know", 'fam': 'family', 'faq': 'frequently asked questions', 'fb': 'facebook', 'fc': 'fingers crossed', 'flex': 'show off', 'ftl': 'for the loss', 'ftw': 'for the win', 'fwiw': "for what it's Worth", 'fyi': 'for your information', 'gal': 'get a life', 'gg': 'good game', 'gl': 'good lu

#**PRE-PROCESSING**

**CASE FOLDING**

Inizialmente, volendo usare `BERT-base-uncased`, avevamo considerato di convertire tutto il testo in minuscolo, un'operazione che il modello avrebbe comunque eseguito durante la tokenizzazione.<br>
Tuttavia, avendo poi optato per `BERT-base-cased` proprio per il suo essere case-sensitive, abbiamo evitato questo pre-processing per preservare distinzioni importanti, come tra "apple" (mela) e "Apple" (azienda).<br>
Un ulteriore vantaggio è che, nel contesto dei social media, il testo in maiuscolo è spesso interpretato come un segnale di enfasi o urla; con il case folding avremmo perso questa informazione.

---

**HASHTAG**

Inizialmente, il nostro obiettivo era quello di segmentare gli hashtag per estrarre il maggior numero possibile di informazioni, poiché lasciarli nella loro forma originale non garantisce una corretta comprensione del loro significato. Per farlo, avevamo previsto di analizzare gli hashtag scorrendoli in entrambi i versi, confrontando le possibili suddivisioni con il vocabolario inglese di `NLTK` e un vocabolario personalizzato di slang e abbreviazioni da noi creato `slang_dict`.

Tuttavia, l'assenza di una struttura standard negli hashtag (come la maiuscola all'inizio di ogni parola) rende difficile una separazione accurata delle parole, con il rischio di alterarne il significato originale. Per questo motivo, abbiamo deciso di mantenere gli hashtag nella loro forma originale presente nel dataset.

Un esempio di criticità è l'hashtag "#todayisagreatday", il quale potrebbe essere esegmentato erroneamente in "today is agre at day", stravolgendone totalmente il significato.

---

**STOPWORDS**

Non abbiamo ritenuto necessaria la rimozione delle stopwords, poiché spesso contribuiscono al significato semantico della frase. In molti casi, la loro eliminazione potrebbe alterare il senso del testo, come accadrebbe con "not happy", che ha un significato opposto rispetto a "happy".

---

**DECODIFICA CARATTERI IN FORMATO HTML**

Abbiamo deciso di decodificare le sequenze di caratteri codificati in formato **HTML entity reference**, come `&amp;` che rappresenta il simbolo `&`. Questo processo converte le entità **HTML** in caratteri leggibili, facilitando l'analisi del testo.

La semplice lettura del dataset con codifica UTF-8 non è sufficiente a sostituire automaticamente questi simboli, poiché vengono trattati come sequenze testuali anziché caratteri speciali. Per questo motivo, abbiamo usato la libreria `html` per applicare questa trasformazione.

In [ ]:
# esempio di tweet con "&" codificato in formato HTML entity reference
print(df_train['text'][11])

# applicazione del pre-processing
df_train['text'] = df_train['text'].apply(html.unescape)
df_val['text'] = df_val['text'].apply(html.unescape)
df_test['text'] = df_test['text'].apply(html.unescape)

# esempio dopo il pre-processing
print(df_train['text'][11])

#NewYork: Several #Baloch &amp; Indian activists hold demonstrations outside @user headquarters demanding Pak to stop exporting #terror into India  
#NewYork: Several #Baloch & Indian activists hold demonstrations outside @user headquarters demanding Pak to stop exporting #terror into India  


**RIMOZIONE DEI TAG "@user"**

Nel dataset, tutti i tag utente sono stati precedentemente normalizzati come `@user`, impedendo loro di influenzare il significato o il contesto dei tweet in cui compaiono. Per questo motivo, abbiamo deciso di rimuoverli.

In aggiunta, rimuoviamo anche "\\n", "\n", "\r" e sostituiamo gli spazi multipli con un singolo spazio.

In [ ]:
def clean_text(text):
    # Rimuove il tag @user
    text = text.replace("@user", "")
    # Sostituisce le sequenze letterali "\n" (backslash + n) e i veri newline (\n e \r) con uno spazio.
    text = re.sub(r'(\\n|\n|\r)', ' ', text)
    # Sostituisce eventuali spazi multipli con uno singolo spazio
    text = re.sub(r'\s+', ' ', text)
    # Rimuove spazi iniziali e finali
    return text.strip()

print(df_train['text'][37])

# Applica la funzione a tutti i DataFrame
df_train['text'] = df_train['text'].apply(clean_text)
df_val['text']   = df_val['text'].apply(clean_text)
df_test['text']  = df_test['text'].apply(clean_text)

print(df_train['text'][37])


her; i want a playful relationship\nme; *kicks her off the couch*  
her; i want a playful relationship me; *kicks her off the couch*


**SLANG E ABBREVIAZIONI**

Queste espressioni, ampiamente utilizzate nei social, possono influenzare l'emozione complessiva del tweet, poiché spesso trasmettono sfumature di significato leggermente diverse rispetto alle loro forme estese. Tuttavia, il modello potrebbe non essere in grado di interpretarle correttamente.

Per questo motivo, abbiamo applicato una trasformazione in cui espandiamo le abbreviazioni e sostituiamo gli slang con una forma più comune, in modo da migliorare l'adattamento del testo al vocabolario del tokenizzatore di `BERT-base-cased`.

Un possibile miglioramento di questo approccio potrebbe essere l'ampliamento del dizionario con ulteriori slang e abbreviazioni.

In [ ]:
def expand_slang(df):
    texts = []
    for text in df:
        splitting = text.split()
        for i in range(len(splitting)):
            stripped = splitting[i].strip(",.!")
            if stripped.lower() in slang_dict:
                splitting[i] = splitting[i].replace(stripped, slang_dict[stripped.lower()])
        texts.append(" ".join(splitting))
    return texts

# esempio di tweet con abbreviazione
print(df_train['text'][4])

# applicazione del pre-processing
df_train['text'] = expand_slang(df_train['text'])
df_val['text'] = expand_slang(df_val['text'])
df_test['text'] = expand_slang(df_test['text'])

# esempio dopo il pre-processing
print(df_train['text'][4])

it's pretty depressing when u hit pan on ur favourite highlighter
it's pretty depressing when you hit pan on your favourite highlighter


**EMOJI ED EMOTICON**

Dopo dei test di encoding e decoding del tokenizer di `BERT-base-cased`, abbiamo riscontrato una quasi totale assenza di token dedicati a emoji ed emoticon.

Per preservarne il significato, abbiamo inizialmente considerato la possibilità di convertirle in forma testuale estesa (ad esempio, "🙄" sarebbe diventato "face with rolling eyes"), sfruttando token già appresi dal modello. Tuttavia, questa soluzione è stata scartata poiché la traduzione estesa non riesce a catturare la stessa espressività delel emoji o delle emoticon.

Abbiamo quindi deciso di aggiungere token specifici per rappresentarle in modo più efficace. Di seguito, le tre opzioni valutate:

1. Utilizzare gli Unicode;
2. Utilizzare direttamente le emoji/emoticon così come appaiono nel dataset;
3. Convertirle in una forma testuale contratta.

1) Questa opzione è stata scartata immediatamente, poiché gli Unicode sono disponibili solo per le emoji. Le emoticon, essendo composte da semplici caratteri testuali, non hanno un codice Unicode e avrebbero richiesto un trattamento separato.

2) Questa soluzione sembrava promettente, ma abbiamo individuato una criticità: le emoticon possono avere più varianti con lo stesso significato. Ad esempio, le due emoticon ":-)" e ":)" rappresentano entrambe una faccia sorridente, ma avrebbero generato token distini nel modello perché composte da caratteri diversi.<br>
Questo problema non sarebbe stato significativo con un dataset di grandi dimensioni, ma la limitatezza del nostro *train set* ci ha portato a scartare anche questa possibilità.

3) Abbiamo scelto di trasformare emoji ed emoticon in una forma testuale contratta, garantendo una maggiore uniformità e riducendo il numero di token distinti.<br>
Ad esempio, "🙄" diventa ":face_with_rolling_eyes:" e ":-)" diventa ":happy_face_smiley:".<br>
Questo approccio consente di unificare alcune traduzioni tra emoji ed emoticon, riducendo la sparsità dei token e migliorando l'apprendimento del modello. Inoltre, avendo meno token distinti che compaiono più frequentemente, il modello ha più opportunità di apprendere il significato emotivo associato a queste espressioni.<br>
Per implementare questa trasformazione, abbiamo utilizzato la libreria `emot`, che fornisce traduzioni testuali nella forma che desideriamo per le emoji, ma in una forma ancora da elaborare per le emoticon; alcune di esse vengono tradotte in più significati (esempio: ":(" viene tradotto in "Frown, sad, andry or pouting"), abbiamo affinato il processo scegliendo:
- La seconda parola quando la traduzione contiene virgole;
- La prima parola quando la traduzione contiene "or".

:( -> Traduzione: "Frown, sad, angry or pouting" -> Scelta finale: `:sad:`<br>
:) -> Traduzione: "Happy face or smiley" -> Scelta finale: `:happy_face:`

In [ ]:
emot_obj = emot.core.emot()

# sostituzione di spazi o "-" con "_" ed aggiunta dei due punti all'inizio e alla fine
def add_signs(emoticon_text):
    emoticon_text = emoticon_text.lower().replace("-", "_").replace(" ", "_")
    return emoticon_text

# le emoticon spesso vengono tradotte come "Frown, sad or pouting". Prendiamo quelle dopo la prima virgola e/o prima di "or"
def clean_emoticon(emoticon_text):
    parts = emoticon_text.split(",")
    if len(parts) > 1:
        emoticon_text = parts[1].strip()
    emoticon_text = emoticon_text.split("or")[0].strip()

    return add_signs(emoticon_text)

# sostituzione di emoji/emoticon con forma testuale e popolamento del dizionario delle traduzioni
def convert_emot_emoji(text, emote_obj, translation_dict):
    # operazioni per le emoji
    for emoji, meaning in zip(emot_obj.emoji(text)['value'], emot_obj.emoji(text)['mean']):
        formatted_meaning = add_signs(meaning)
        text = text.replace(emoji, f" {formatted_meaning} ")
        translation_dict[emoji] = formatted_meaning
    # operazioni per le emoticon
    for emoticon, meaning in zip(emot_obj.emoticons(text)['value'], emote_obj.emoticons(text)['mean']):
        formatted_meaning = clean_emoticon(meaning)
        formatted_meaning = ":" + formatted_meaning + ":"
        text = text.replace(emoticon, f" {formatted_meaning} ")
        translation_dict[emoticon] = formatted_meaning
    return text

# dizionario in cui salviamo le traduzioni effettuate
translation_dict = {}

# oggetto per catturare le occorrenze di emoji ed emoticon
emot_obj = emot.core.emot()

# esempio nel dataset con emoji ed emoticon
print(df_train['text'][32])

# applicazione del pre-processing ("lambda" serve per passare al metodo tutti e tre i parametri e non solo la colonna)
df_train['text'] = df_train['text'].apply(lambda text: convert_emot_emoji(text, emot_obj, translation_dict))

# esempio dopo il pre-processing
print(df_train['text'][32])

'shit' doesn't even begin to describe these fiery little demons straight from hell 🌝🌚 ;)
'shit' doesn't even begin to describe these fiery little demons straight from hell  :full_moon_face:  :new_moon_face:   :wink: 


Per il pre-processing di emoji ed emoticon nel `test set` e nel `validation set`, abbiamo adottato una strategia specifica. Poiché le nuove forme testuali aggiunte al tokenizzatore possono essere ricavate solo dalle occorrenze presenti nel `train set`, abbiamo costruito un dizionario contenente esclusivamente le trasformazioni effettuate in fase di training.

Nel pre-processing del `test set` e del `validation set`, utilizziamo questo dizionario per sostituire le emoji ed emoticon già riscontrate. Tutte le occorrenze non presenti nel `train set` vengono invece traformate in una forma testuale *descrittiva estesa*, in modo da preservarne il significato ed evitare la generazioen di token `[UNK]`.

Ad esempio, se ":)" non fosse stata incontrata nel `train set`, verrebbe trasformata in "happy face".

In [ ]:
def translate_from_dict(text, translation_dict):
    for key, value in translation_dict.items():
        value = " " + value + " "
        text = text.replace(key, value)
    return text

def remove_signs(emoji_text):
    emoji_text = emoji_text.replace("_", " ").replace("-", " ").replace(":", "")
    return emoji_text

def choose_meaning(emoticon_text):
    parts = emoticon_text.split(",")
    if len(parts) > 1:
        emoticon_text = parts[1].strip()
    emoticon_text = emoticon_text.split("or")[0].strip()
    return emoticon_text

def extend_emo(text, emot_obj):
    # operazioni per le emoji
    for emoji, meaning in zip(emot_obj.emoji(text)['value'], emot_obj.emoji(text)['mean']):
        formatted_meaning = remove_signs(meaning)
        text = text.replace(emoji, f" {formatted_meaning} ")
    # operazioni per le emoticon
    for emoticon, meaning in zip(emot_obj.emoticons(text)['value'], emot_obj.emoticons(text)['mean']):
        chosen_meaning = choose_meaning(meaning)
        text = text.replace(emoticon, f" {chosen_meaning} ")
    return text.lower()


# traduzione di emoji/emoticon appresi dal dataset di train
df_val['text'] = df_val['text'].apply(lambda text: translate_from_dict(text, translation_dict))
df_test['text'] = df_test['text'].apply(lambda text: translate_from_dict(text, translation_dict))

# traduzione di emoji/emoticon non presenti nel dataset di train in forma estesa per evitare token [UNK]
df_val['text'] = df_val['text'].apply(lambda text: extend_emo(text, emot_obj))
df_test['text'] = df_test['text'].apply(lambda text: extend_emo(text, emot_obj))

#**AGGIUNTA DI NUOVI TOKEN**

Per effettuare il **pre-training** sul nostro dataset, abbiamo aggiunto al tokenizzatore i nuovi token corrispondenti alle traduzioni salvate come valori all'interno del dizionario `translation_dict`.

Nel `train set`, abbiamo riscontrato 192 emoji ed emoticon uniche (su un totale di oltre 900). Di queste, alcune sono state accorpate poiché condividevano la stessa traduzione, portando così all'aggiunta di 183 nuovi token.

In [ ]:
translation_dict
lunghezza_max_token = max(len(value) for value in translation_dict.values())

In [ ]:
# creazione della lista delle traduzioni di emoji/emoticon
emo_texts = []
for item in translation_dict:
    if translation_dict[item] not in emo_texts:
        emo_texts.append(translation_dict[item])

print(len(emo_texts))

# controllo del numero di token iniziali di BERT
print("The tokenizer had", len(tokenizer))

# aggiunta dei nuovi token al tokenizzatore del modello
num_added_tokens = tokenizer.add_tokens(emo_texts)
print("Added", num_added_tokens, "tokens")

# ridimensionamento del modello in base alla nuova dimensione del tokenizer
model_lm.resize_token_embeddings(len(tokenizer))

183
The tokenizer had 28996
Added 183 tokens


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(29179, 768, padding_idx=0)

In [ ]:
sample_text = "This is a :face_with_rolling_eyes: example."

# Tokenizza il testo
encoded = tokenizer(sample_text)

# Visualizza i token
print("Tokenized text:", encoded)

Tokenized text: {'input_ids': [101, 1188, 1110, 170, 28997, 1859, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# Verifichiamo la dimensione del vocabolario dopo l'aggiunta dei nuovi token
print("Dimensione aggiornata del vocabolario del tokenizer:", len(tokenizer))

# Verifichiamo la dimensione della matrice di embedding nel modello
print("Dimensione della matrice di embedding del modello:", model_lm.get_input_embeddings().weight.size(0))

Dimensione aggiornata del vocabolario del tokenizer: 29179
Dimensione della matrice di embedding del modello: 29179


#**DATA AUGMENTATION**

In [ ]:
class_counts = df_train['labels'].value_counts()
print(class_counts)

labels
0    1400
3     855
1     708
2     294
Name: count, dtype: int64


La **Data Augmentation** è necessaria in questo caso per bilanciare il dataset. Come evidenziato dalla distribuzione delle classi c'è uno squilibrio significativo, il che può portare ad un modello che non effettua correttamente la classificazione delle classi sottorappresentate, in quando tende ad apprendere maggiormente dalle classi dominanti.

Abbiamo deciso di applicare più tecniche per poter creare dei nuovi campioni artificiali, in modo che possano essere eterogenei tra di loro e rispetto ai campioni originali. Abbiamo utilizzato le seguenti tecniche:
- Backtranslation
- Synonyms Replacement
- Random Deletion
- Typo Simulation
- Paraphrase Generation

##**BACKTRANSLATION**

Questo codice implementa un sistema per la traduzione automatica e la **Backtranslation** di testi, con un'attenzione particolare alla gestione delle emoji ed emoticon.<br>
Dopo aver installato la libreria `easyNMT` ed effettuato gli import necessari, abbiamo definito una funzione per liberare la memoria della GPU, utile per garantire un utilizzo efficiente delle risorse durante l'elaborazione.<br>
La funzione `replace_emoji_with_placeholder` si occupa della sostituzione delle emoji ed emoticon all'interno di un testo con dei segnaposto.<br>
In seguito alla **Backtranslation**, la funzione `restore_placeholders` permette di ripristinare le emoji ed emoticon originali.<br>
Poi, definiamo la classe `BacktranslateDataset`, progettata per gestire un dataset di testi, in modo da processare più tweet alla volta, ottimizzando le operazioni. All'interno di questa classe, viene utilizzato il modello `EasyNMT` con la variante `mbart50_m2m`, che supporta traduzioni tra più lingue.

In [ ]:
!pip install easyNMT
import torch
from easynmt import EasyNMT

nltk.download('punkt_tab')
nltk.download('punkt')

# Funzione per liberare la memoria GPU
def release_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()  # Assicuriamo che tutte le operazioni siano completate


# Funzione per sostituire le emoji e le emoticon con dei placeholder
def replace_emoji_with_placeholder(text):
    # Pattern per catturare stringhe come :happy_face: o :sad-face:
    pattern = rf":(?=.{{1,{lunghezza_max_token - 2}}}:)(\w+(?:-\w+)*):"
    mapping = {}

    # Sostituzione dell'occorrenza di un token token di emoji ed emoticon con un placeholder
    def repl(match):
        original = match.group(0)
        index = len(mapping)
        placeholder = f"__EMOJI{index}__"
        mapping[placeholder] = original
        return placeholder

    new_text = re.sub(pattern, repl, text)
    return new_text, mapping

# Sostituzione inversa del placeholder con l'emoji/emoticon originale
def restore_placeholders(text, mapping):
    for placeholder, original in mapping.items():
        new_text = text.replace(placeholder, original)
        if new_text != text:
          text = new_text
    return text


# Creazione dei batch da processare e applicazione della Backtranslation
class BacktranslateDataset(Dataset):
    def __init__(self, dataframe, mid_lang='fr'):
        self.data = dataframe
        self.mid_lang = mid_lang
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = EasyNMT('mbart50_m2m', device=device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data.iloc[index]['text']
        return text

    def collate_fn(self, batch):
        processed_texts = []
        mappings = []

        # Sostituzione con placeholder per ogni testo
        for text in batch:
            new_text, mapping = replace_emoji_with_placeholder(text)
            processed_texts.append(new_text)
            mappings.append(mapping)

        # Traduzione in lingua intermedia (default francese)
        translated = self.model.translate(processed_texts, source_lang='en', target_lang=self.mid_lang)

        # Traduzione di ritorno in inglese (backtranslation)
        backtranslated = self.model.translate(translated, source_lang=self.mid_lang, target_lang='en')

        # Ripristiniamo le emoji originali in ciascun testo tradotto
        restored = []
        for text, mapping in zip(backtranslated, mappings):
            restored_text = restore_placeholders(text, mapping)
            restored.append(restored_text)

        # Liberiamo la memoria GPU alla fine dell'elaborazione del batch
        release_gpu_memory()
        return restored

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**ESEMPIO DI BACKTRANSLATION**

In [ ]:
df_train['text'][0]

"“Worry is a down payment on a problem you may never have'. Joyce Meyer. #motivation #leadership #worry"

In [ ]:
df_train_new = df_train.iloc[0:1, :]
# Creiamo il dataset
dataset = BacktranslateDataset(df_train_new, mid_lang='fr')

# Creiamo il DataLoader, specificando la funzione collate_fn definita nella classe
dataloader = DataLoader(dataset, batch_size=32, collate_fn=dataset.collate_fn, shuffle=False)

translated_texts = []
for batch in dataloader:
    translated_texts.extend(batch)
translated_texts

Fine primo batch


['"Worry is a rebate for a problem you may never have had." Joyce Meyer. # motivation # leadership # concern']

**CARICAMENTO DEL DATAFRAME CONTENENTE I TWEET POST BACKTRANSLATION**

Dato il grande impiego di tempo per effettuare la **Backtranslation**, abbiamo deciso di applicarla a tutto il dataset di *train*, in modo da caricare le versioni "Augmented" nella nuova colonna 'translated' del DataFrame `df_train`.

In [ ]:
translated = pd.read_csv('/content/drive/MyDrive/Progetto NLP/Claudio/df_train_translated.csv')['translated'].tolist()
#Aggiungiamo la colonna "translated" al dataframe df_train
df_train['translated'] = translated

In [ ]:
df_train

,text,labels,translated
0,“Worry is a down payment on a problem you may ...,2,"""Worry is a rebate for a problem you may never..."
1,My roommate: it's okay that we can't spell bec...,0,My roommate: it's good that we can't spell bec...
2,No but that's so cute. Atsu was probably shy a...,1,"No, but it's so fun. Atsu was probably shy abo..."
3,Rooneys fucking untouchable isn't he? Been fuc...,0,"Rooneys fucking untouchable, right? I was unfo..."
4,it's pretty depressing when you hit pan on you...,3,It's pretty depressing when you touch the pan ...
...,...,...,...
3252,I get discouraged because I try for 5 fucking ...,3,I am discouraged because I tried for 5 years o...
3253,The are in contention and hosting nation and C...,3,They are in conflict and the host nation and C...
3254,"as a fellow UP grad, i shiver at the shallowne...",0,"As a graduate colleague from UP, I have tremor..."
3255,You have a #problem? Yes! Can you do #somethin...,0,Do you have a # problem? Yes! Can you do somet...


##**SYNONYMS REPLACEMENT**

Il codice ha lo scopo di sostituire alcune parole in una frase con i loro sinonimi.<br>
La funzione principale, `replace_with_synonyms` , utilizza il tokenizer di **BERT** per tokenizzare la frase di input, trasformandola in una lista di token. Successivamente, calcola un numero casuale di parole da sostituire in base alla lunghezza della frase.<br>
Filtra le parole da modificare, escludendo:
- Parole emotive (positive o negative) per preservare il significato emotivo della frase, individuate utilizzando il lessico delle opinioni di NLTK;
- Parole di una sola lettera, poiché hanno un significato semantico limitato;
- Token che iniziano con "#" o ":" (o sono preceduti da "#") per evitare errori nel trattamento di emoji, emoticon ed hashtag.

Una volta filtrate le parole candidate alla sostituzione, seleziona un sottoinsieme casuale di queste e utilizza la funzione `get_synonyms` per trovare sinonimi di ciascuna parola. Se non ci sono sinonimi disponibili, si mantiene la parola originale, garantendo la coerenza del testo.<br>
Un dizionario associa ogni parola selezionata al suo sinonimo, e viene sostituita solo la sua prima occorrenza. Infine, i token aggiornati vengono convertiti nuovamente in una frase tramite il tokenizer di **BERT**.

L'uso del tokenizer del modello che stiamo usando è cruciale per gestire correttamente emoji ed emoticon, che vengono trattate come entità uniche (ad esempio `:smiling_face_with_heart_eyes:`) dato che sono state precedentemente aggiunte allo stesso tokenizer. Questo approccio aiuta a preservare il significato semantico delle emoji ed emoticon e minimizza gli errori di sostituzione.

In [ ]:
import random
from nltk.corpus import opinion_lexicon
from nltk.corpus import wordnet as wn
nltk.download('opinion_lexicon')
nltk.download('wordnet')

# Carichiamo le parole emozionali di NLTK per evitarne la modifica
positive_words = set(opinion_lexicon.positive())
negative_words = set(opinion_lexicon.negative())
sentiment_words = positive_words.union(negative_words)

# Funzione per recuperare il sinonimo legato alla parola data
def get_synonyms(word):
    synonyms = set()
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            if lemma.name().lower() != word.lower():
                synonyms.add(lemma.name().replace("_", " "))
    return list(synonyms)

# Funzione principale di Synonyms Replacement
def replace_with_synonyms(sentence, tokenizer):
    # Tokenizza la frase con il tokenizer di BERT
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    decoded_sentence = tokenizer.decode(token_ids)
    tokens = tokenizer.tokenize(decoded_sentence)

    # Determiniamo il numero di sostituzioni in base alla lunghezza della frase
    if len(tokens) <= 5:
        num_replacements = 1
    elif len(tokens) <= 10:
        num_replacements = random.randint(2, 3)
    elif len(tokens) <= 20:
        num_replacements = random.randint(3, 4)
    else:
        num_replacements = random.randint(4, 5)

    # Selezione delle parole che si possono sostituire con il sinonimo
    candidate_words = [
        word for i, word in enumerate(tokens)
        if word.lower() not in sentiment_words
        and len(word) > 1
        and not word.startswith("#")
        and not word.startswith(":")
        and (i == 0 or tokens[i - 1] != "#")
    ]

    # Se non ci sono parole sostituibili restituiamo la frase originale
    if not candidate_words or num_replacements == 0:
        return sentence

    # Selezione delle parole da sostituire
    words_to_replace = random.sample(list(set(candidate_words)), min(num_replacements, len(set(candidate_words))))

    # Crea un dizionario {parola_originale : sinonimo_casuale}
    replacement_dict = {}
    for word in words_to_replace:
        synonyms = get_synonyms(word)
        if synonyms:  # Se ci sono sinonimi disponibili
            replacement_dict[word] = random.choice(synonyms)
        else:
            replacement_dict[word] = word  # Se non ci sono sinonimi, mantiene la parola originale

    # Sostituiamo solo la prima occorrenza della parola nella lista dei tokens
    new_tokens = []
    for word in tokens:
        if word in replacement_dict:  # Se la parola è in replacement_dict
            # Se è la prima volta che incontriamo questa parola da sostituire
            new_tokens.append(replacement_dict[word])
            replacement_dict.pop(word, None)  # Rimuoviamo la parola dalla lista di sostituzioni, così non verrà sostituita di nuovo
        else:
            new_tokens.append(word)

    # Ricostruiamo la frase
    new_sentence = tokenizer.convert_tokens_to_string(new_tokens)

    return new_sentence

[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


**ESEMPIO DI SYNONYMS REPLACEMENT**

In [ ]:
sentence = "Follow this amazing Australian author #fiction #horror #zombies #angels #demons #vampires #werewolves #follow #authorlove"

In [ ]:
new_sentence = replace_with_synonyms(sentence, tokenizer)
print("Modified sentence:", new_sentence)

Modified sentence: comply this amazing Aussie generator # fiction # horror # zombies # angels # demons # vampires # werewolves # follow # authorlove


##**RANDOM DELETION**

Il codice ha l'obiettivo di rimuovere un numero variabile di parole da una frase, preservandone la coerenza generale.

La funzione `determine_num_removals` decide il numero di parole da rimuovere in base alla lunghezza della frase. Le regole sono:
- Se la frase a 5 o meno parole, viene rimossa una sola parola;
- Se la frase contiene tra 6 e 10 parole, viene rimosso un numero casuale tra 1 e 2 parole;
- Se la frase ha tra 11 e 20 parole, il numero di parole rimosse varia tra 2 e 3;
- Per frasi più lunghe di 20 parole, vengono rimosse da 3 a 4 parole.

La funzione `remove_words` inizia tokenizzando la frase tramite il tokenizer di **BERT** per i motivi già spiegati per la **Synonyms Replacement**. Successivamente, la frase viene decodificata per ottenere una lista di token rappresentativi del testo originale. Viene applicato un filtro per identificare le parole candidabili alla rimozioni (come visto per la **Synonyms Replacement**).<br>
Una volta identificati i token rimovibili, la funzione seleziona casualmente un numero di parole da rimuovere, in base alla lunghezza determinata da `determine_num_removals`. Queste parole vengono rimosse dalla lista dei token. Infine, la frase viene ricostruita convertendo i token rimanenti in un testo leggibile utilizzando sempre il tokenizer di **BERT**.

In [ ]:
# Funzione per determinare il numero di rimozioni in base alla lunghezza del tweet
def determine_num_removals(sentence_length):
    if sentence_length <= 5:
        return 1
    elif sentence_length <= 10:
        return random.randint(1, 2)
    elif sentence_length <= 20:
        return random.randint(2, 3)
    else:
        return random.randint(3, 4)

# Funzione principale di Random Deletion
def remove_words(sentence, tokenizer):
    # Tokenizza la frase con il tokenizer di BERT
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    decoded_sentence = tokenizer.decode(token_ids)
    tokens = tokenizer.tokenize(decoded_sentence)

    sentence_length = len(tokens)

    # Usa la funzione determine_num_removals per decidere il numero di rimozioni
    num_removals = determine_num_removals(sentence_length)

    # Selezione delle parole che si possono rimuovere
    removable_words = [
        word for i, word in enumerate(tokens)
        if word.lower() not in sentiment_words
        and len(word) > 1
        and not word.startswith("#")
        and not word.startswith(":")
        and (i == 0 or tokens[i - 1] != "#")
    ]
    if not removable_words:  # Se nessuna parola è rimovibile, restituiamo la frase originale
        return sentence

    new_tokens = tokens[:]  # Creiamo una copia della frase da modificare

    for _ in range(num_removals):
        if not removable_words:  # Se non ci sono più parole rimovibili, si esce dal loop
            break

        word_to_remove = random.choice(removable_words)   # Selezioniamo una parola da rimuovere
        removable_words.remove(word_to_remove)            # Rimuoviamo la parola scelta per evitare doppie rimozioni


        new_tokens.remove(word_to_remove)  # Rimuoviamo la parola dalla lista dei token

    # Ricostruiamo la frase
    new_sentence = tokenizer.convert_tokens_to_string(new_tokens)
    return new_sentence


**ESEMPIO DI RANDOM DELETION**

In [ ]:
sentence = "Follow this amazing Australian author #fiction #horror #zombies #angels #demons #vampires #werewolves #follow #authorlove"
new_sentence = remove_words(sentence, tokenizer)
print("Modified sentence:", new_sentence)

Modified sentence: amazing author # fiction # horror # zombies # angels # demons # vampires # werewolves # follow # authorlove


##**TYPO SIMULATION**

Il seguente codice ha lo scopo di introdurre modifiche casuali a una frase simulando errori di battitura (typos) o omissioni di caratteri, allo scopo di arricchire i dati con varianti realistiche delle frasi originali.

Il dizionario `keyboard_adjacent` mappa ogni lettera a un insieme di lettere vicine sulla tastiera QWERTY.

La funzione `determine_num_perturbations` calcola il numero di modifiche da applicare alla frase in base alla sua lunghezza.

La funzione `introduce_typo` simula un errore di battitura, sostituendo un carattere di una parola con un carattere vicino, non modificando parole troppo corte (≤ 3 caratteri).

La funzione `omit_character` rimuove casualmente un carattere da una parola. Come per il metodo precedente, non modifica parole troppo corte.

La funzione `perturbation` tokenizza la frase con il tokenizer di **BERT** per gestire simboli, subwords e parole in modo accurato. Ha un funzionamento simile a metodi visti per altre celle di codice, per preservare il significato semantico delle frasi e i simboli speciali (hashtag, emoji ed emoticon). Seleziona casualmente un sottoinsieme di parole modificabili e applica a ciascuna di esse una delle due modifiche disponibili. Infine, ricostruisce la frase modificata.

In [ ]:
# Dizionario di lettere vicine sulla tastiera QWERTY
keyboard_adjacent = {
    "a": "qwsz", "b": "vghn", "c": "xdfv", "d": "ersfxc",
    "e": "wrsd", "f": "ertdgvc", "g": "rtyfhvb", "h": "tyugjnb",
    "i": "uojk", "j": "uihkmn", "k": "ioljm", "l": "opk",
    "m": "njk", "n": "bhjm", "o": "ipkl", "p": "ol",
    "q": "wa", "r": "etdf", "s": "awedxz", "t": "rfgby",
    "u": "yihj", "v": "cfgb", "w": "qase", "x": "zsdc",
    "y": "tghu", "z": "asx"
}

# Funzione per determinare il numero di caratteri da sostituire in base alla lunghezza del tweet
def determine_num_perturbations(sentence_length):
    if sentence_length <= 5:
        return 1
    elif sentence_length <= 10:
        return random.randint(2, 3)
    elif sentence_length <= 20:
        return random.randint(3, 4)
    else:
        return random.randint(4, 5)

# Funzione per introdurre l'errore di battitura
def introduce_typo(word):
    # Evitiamo di modificare parole molto corte
    if len(word) <= 3:
        return word

    # Sostituzione del carattere
    index = random.randint(0, len(word) - 1)
    if word[index] in keyboard_adjacent:
        typo_char = random.choice(keyboard_adjacent[word[index]])
        word = word[:index] + typo_char + word[index + 1:]
    return word

# Funzione per rimuovere un carattere
def omit_character(word):
    # Evitiamo di modificare parole molto corte
    if len(word) <= 3:
        return word

    # Rimozione del carattere
    index = random.randint(0, len(word) - 1)
    word = word[:index] + word[index + 1:]
    return word


# Funzione principale di Typo Simulation
def perturbation(sentence, tokenizer):
    # Tokenizza la frase con il tokenizer di BERT
    tokens = tokenizer.tokenize(sentence)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    decoded_sentence = tokenizer.decode(token_ids)
    tokens = tokenizer.tokenize(decoded_sentence)

    sentence_length = len(tokens)
    num_perturbations = determine_num_perturbations(sentence_length)

    if sentence_length <= 1:  # Se la frase è troppo corta restituiamo il tweet originale
        return sentence

    # Selezione delle parole che si possono rimuovere
    non_sentiment_words = [
            word for i, word in enumerate(tokens)
            if word.lower() not in sentiment_words
            and len(word) > 3
            and not word.startswith("#")
            and not word.startswith(":")
            and (i == 0 or tokens[i - 1] != "#")
        ]

    if not non_sentiment_words:  # Se non ci sono parole modificabili restituiamo il tweet
        return sentence

    # Selezione delle parole da sostituire
    words_to_modify = random.sample(non_sentiment_words, min(num_perturbations, len(non_sentiment_words)))
    modified_words = set()

    # Applicazione della sostituzione o rimozione in modo casuale
    for i in range(len(tokens)):
        if tokens[i] in words_to_modify and tokens[i] not in modified_words:
            modified_words.add(tokens[i])  # Aggiungiamo la parola al set delle parole modificate
            perturbation_type = random.choice(["typo", "omit"])
            if perturbation_type == "typo":
                tokens[i] = introduce_typo(tokens[i])
            elif perturbation_type == "omit":
                tokens[i] = omit_character(tokens[i])

    new_sentence = tokenizer.convert_tokens_to_string(tokens)
    return new_sentence



**ESEMPIO DI TYPO SIMULATION**

In [ ]:
sentence = "Follow this amazing Australian author #fiction #horror #zombies #angels #demons #vampires #werewolves #follow #authorlove"
new_sentence = perturbation(sentence, tokenizer)
print("Modified sentence:", new_sentence)

Modified sentence: Foolow tuis amazing Austealian ahthor # fiction # horror # zombies # angels # demons # vampires # werewolves # follow # authorlove


##**PARAPHRASE GENERATION**

Il seguente codice utilizza il modello di parafrasi pre-addestrato `humarin/chatgpt_paraphraser_on_T5_base` per generare versioni parafrasate di frasi di input. Grazie all'uso di un `DataLoader`, il processo è ottimizzato per gestrire dataset di grandi dimensioni suddividendoli in batch.

La funzione `generaFrasiBatch` elabora un `DataLoader` contenente batch di frasi da parafrasare. Per ogni batch:

- Le frasi vengono convertite in tensori tokenizzati con padding e troncamento a una lunghezza massima di 512 token. I tensori vengono spostati sulla GPU;
- Il modello genera versioni parafrasate con la funzione `generate`. Si introduce variazione nel processo di parafrasi tramite l'argomento `do_sample=True`;
- Gli output del modello (tensori) vengono decodificati in testo leggibile, saltando i token speciali. Le frasi parafrasate vengono aggiunte a una lista globale;
- I tensori intermedi vengono eliminati per liberare memoria.

La funzione restituisce una lista contenente tutte le frasi parafrasate che sono state generate dal modello.

In [ ]:
class GenerateSentences(Dataset):
    def __init__(self, series):
        self.data = series

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        if isinstance(index, (list, np.ndarray, pd.Index)):
            # Se viene passato un batch di indici, restituisce un batch di frasi come lista
            return self.data.iloc[index].tolist()
        else:
            # Se viene passato un singolo indice, restituisce una singola frase
            return self.data.iloc[index]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Carichiamo il modello e il tokenizer
model_paraphrase_name = "humarin/chatgpt_paraphraser_on_T5_base"
tokenizer_paraphrase = AutoTokenizer.from_pretrained(model_paraphrase_name)
model_paraphrase = AutoModelForSeq2SeqLM.from_pretrained(model_paraphrase_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_paraphrase.to(device)

def generaFrasiBatch(dataloader):

    # Lista per salvare i risultati
    paraphrased_texts = []

    # Elaboriamo un batch alla volta
    for batch in dataloader:
        # Tokenizziamo l'input e spostiamo i tensori su GPU
        inputs = tokenizer_paraphrase(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generiamo i testi parafrasati
        outputs = model_paraphrase.generate(**inputs, max_length=100, do_sample=True)

        # Decodifichiamo ogni output e aggiungiamo alla lista
        paraphrased_batch = [tokenizer_paraphrase.decode(output, skip_special_tokens=True) for output in outputs]
        paraphrased_texts.extend(paraphrased_batch)

        # Dopo aver processato il batch:
        del inputs, outputs       # Eliminiamo i tensori per liberare memoria
        torch.cuda.empty_cache()  # Libera la cache della GPU
    return paraphrased_texts

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**ESEMPIO DI PARAPHRASE GENERATION**

In [ ]:
paraphrased_texts = []
df_new = df_train.iloc[0:1, 0]
dataset = GenerateSentences(df_new)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
paraphrased_texts = generaFrasiBatch(dataloader)
dataAugmented_1 = pd.Series(paraphrased_texts)

In [ ]:
df_new[0]

"“Worry is a down payment on a problem you may never have'. Joyce Meyer. #motivation #leadership #worry"

In [ ]:
dataAugmented_1[0]

'Joyce Meyer argued that worry is a small price to pay to put a problem out of reach.'

##**APPLICAZIONE DEI METODI DI DATA AUGMENTATION**

Raggruppiamo i campioni del dataset in base alla classe, in modo da poter applicare le tecniche di **Data Augmentation** distintamente, gestendo opportunamente la quantità di dati da generare

In [ ]:
df_original = df_train[['text', 'labels']].copy()

tweetByClass = df_train.groupby('labels')

tweetByClass.head()

class1 = tweetByClass.get_group('1')  # joy
class2 = tweetByClass.get_group('2')  # optimism
class3 = tweetByClass.get_group('3')  # sadness
class4 = tweetByClass.get_group('0')  # anger

print("class1 has: ", len(class1), " elements \t example: ", class1.iloc[0])
print("class2 has: ", len(class2), " elements \t example: ", class2.iloc[0])
print("class3 has: ", len(class3), " elements \t example: ", class3.iloc[0])

CampioniClasseMaggiore = len(class4)
print(f'Campioni classe con più elementi (anger) --> {len(class4)}')

listaClassi = [class1, class2, class3]

class1 has:  708  elements 	 example:  text          No but that's so cute. Atsu was probably shy a...
labels                                                        1
translated    No, but it's so fun. Atsu was probably shy abo...
Name: 2, dtype: object
class2 has:  294  elements 	 example:  text          “Worry is a down payment on a problem you may ...
labels                                                        2
translated    "Worry is a rebate for a problem you may never...
Name: 0, dtype: object
class3 has:  855  elements 	 example:  text          it's pretty depressing when you hit pan on you...
labels                                                        3
translated    It's pretty depressing when you touch the pan ...
Name: 4, dtype: object
Campioni classe con più elementi (anger) --> 1400


Abbiamo deciso di applicare tutte e cinque le tecniche di Data Augmentation alle classi meno popolose, in modo da creare dei campioni "sintetici" più eterogenei possibili.<br>
Prima di applicare gli split e l'augmentation selezioniamo solo il 50% dei campioni per classe.
Per le classi *joy* e *sadness* che hanno rispettivamente 708 e 855 campioni, abbiamo deciso di applicare le tecniche a determinate percentuali di campioni:
- generazioni frasi -> 30%
- backtranslation -> 30%
- sostituzione con sinonimi -> 15%
- rimozione parole -> 15%
- perturbazione -> 10%


Questa scelta è stata fatta per applicare a porzioni sempre più piccole quelle trasformazioni che più probabilmente possono portare all'aggiunta di rumore o alla distorsione del significato originale del testo.<br>
Per la classe *optimism*, composta da 294 campioni, abbiamo deciso di applicare tutte le trasformazioni al 50%  dei campioni, per cercare di avvicinare il numero di campioni complessivi a quelli contenuti nella classe *anger* (la più popolata).

In [ ]:
dizionario_etichette = {0 : 'anger', 1 : 'joy', 2 : 'optimism' , 3 :'sadness'}

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Data Augmentation
def SplitDataAugmentation(listaClassi, df_original):

  for i in listaClassi:
    classe = dizionario_etichette[int(i['labels'].iloc[0])]
    # Se la classe è optimism (2) utilizza l'altra funzione
    if(classe == 'optimism'):
      df_original = dataAugmentedClasse2(i, df_original)
    else:
      #joy and sadness
      campioniDaGenerare = CampioniClasseMaggiore - len(i)
      size = len(i) // 2

      first30 = size * 3 // 10                    # Primo 30% Generazione di frasi
      second30 = first30 + size * 3 // 10         # Secondo 30%  Backtranslation
      first15 = second30 + int(size * 1.5 // 10)  # Successivo 15% Synonym replacement
      second15 = first15 + int(size * 1.5 // 10)  # Successivo 15% random deletion
      last10 = size                               # Rimanente 10%  perturbation
      print(first30, second30, first15, second15, last10)

      # Estraiamo le percentuali di campioni dalla lista
      split_0_30 = i['text'].iloc[:first30]                 # Paraphrase Generation
      split_30_60 = i['translated'].iloc[first30:second30]  # Backtranslation
      split_60_75 = i['text'].iloc[second30:first15]        # Synonyms Replacement
      split_75_90 = i['text'].iloc[first15:second15]        # Random Deletion
      split90_100 = i['text'].iloc[second15:last10]         # Typo Simulation

      # Applichiamo le operazioni di Data Augmentation
      dataAugmented_1 = split_0_30
      dataset = GenerateSentences(dataAugmented_1)
      dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
      paraphrased_texts = generaFrasiBatch(dataloader)

      dataAugmented_1 = pd.Series(paraphrased_texts, index=split_0_30.index)
      dataAugmented_2 = split_30_60
      dataAugmented_3 = split_60_75.apply(lambda sentence: replace_with_synonyms(sentence, tokenizer))
      dataAugmented_4 = split_75_90.apply(lambda sentence: remove_words(sentence, tokenizer))
      dataAugmented_5 = split90_100.apply(lambda sentence: perturbation(sentence, tokenizer))

      # Uniamo tutti i campioni "sintetici" in un un'unica lista
      global_augmented = pd.concat([
        dataAugmented_1,
        dataAugmented_2,
        dataAugmented_3,
        dataAugmented_4,
        dataAugmented_5
      ], ignore_index=True)

      df_new = pd.DataFrame({
              'text': global_augmented,
              'labels': [(i['labels'].iloc[0])] * len(global_augmented)
          })
      print(f'Lunghezza df_new = {len(df_new)}')
      print(f'Lunghezza df_original prima della concatenazione -> {len(df_original)}')
      df_original = pd.concat([df_original, df_new], ignore_index=True)
      print(f'Lunghezza df_original dopo la concatenazione -> {len(df_original)}')

  return df_original


# Data Augmentation per la classe 2
def dataAugmentedClasse2(class2, df_original):
  dataAugmented_1 = class2['text'].copy()
  dataAugmented_1 = dataAugmented_1.iloc[:len(dataAugmented_1) // 2]
  dataset = GenerateSentences(dataAugmented_1)
  dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
  paraphrased_texts = generaFrasiBatch(dataloader)
  dataAugmented_1 = pd.Series(paraphrased_texts)

  dataAugmented_2 = class2['translated'].copy()
  dataAugmented_2 = dataAugmented_2.iloc[:len(dataAugmented_2) // 2]
  dataAugmented_3 = class2['text'].apply(lambda sentence: replace_with_synonyms(sentence, tokenizer))
  dataAugmented_3 = dataAugmented_3.iloc[:len(dataAugmented_3) // 2]
  dataAugmented_4 = class2['text'].apply(lambda sentence: remove_words(sentence, tokenizer))
  dataAugmented_4 = dataAugmented_4.iloc[:len(dataAugmented_4) // 2]
  dataAugmented_5 = class2['text'].apply(lambda sentence: perturbation(sentence, tokenizer))
  dataAugmented_5 = dataAugmented_5.iloc[:len(dataAugmented_5) // 2]

  global_augmented = pd.concat([
        dataAugmented_1,
        dataAugmented_2,
        dataAugmented_3,
        dataAugmented_4,
        dataAugmented_5
      ], ignore_index=True)

  df_new = pd.DataFrame({
              'text': global_augmented,
              'labels': [(class2['labels'].iloc[0])] * len(global_augmented)
          })

  return pd.concat([df_original, df_new], ignore_index=True)


df_updated = SplitDataAugmentation(listaClassi, df_original)

Campioni da generare per classe joy = 692
106 212 265 318 354
Lunghezza df_new = 354
Lunghezza df_original prima della concatenazione -> 3257
Lunghezza df_original dopo la concatenazione -> 3611
Campioni da generare per classe sadness = 545
128 256 320 384 427
Lunghezza df_new = 427
Lunghezza df_original prima della concatenazione -> 4346
Lunghezza df_original dopo la concatenazione -> 4773


**SALVATAGGIO DEL NUOVO DATASET**

In [ ]:
df_updated.to_csv(f"/content/drive/MyDrive/Progetto NLP/Claudio/df_train_halfaugmented.csv", index=False)

**CARICAMENTO DEL NUOVO DATASET**

In [ ]:
path = "/content/drive/MyDrive/Progetto NLP/Claudio/df_train_halfaugmented.csv"
df_updated = pd.read_csv(path)


In [ ]:
df_updated

,text,labels
0,“Worry is a down payment on a problem you may ...,2
1,My roommate: it's okay that we can't spell bec...,0
2,No but that's so cute. Atsu was probably shy a...,1
3,Rooneys fucking untouchable isn't he? Been fuc...,0
4,it's pretty depressing when you hit pan on you...,3
...,...,...
4768,"you maie my heat shake, ben and break. '",3
4769,A jight wher depression is winning... # depres...,3
4770,# Terencecutcher # Tulsa the man onthe helicop...,3
4771,xactly what I hzve been saing on facebook..,3


In [ ]:
class_counts = df_updated['labels'].value_counts()
print(class_counts)

labels
0    1400
3    1282
1    1062
2    1029
Name: count, dtype: int64


**CONTROLLI SULLE LUNGHEZZE DEI TWEET**

Abbiamo anche ricercato il più lungo (per assicurarci di non eccedere il limite di token che può accettare in ingresso il modello), il più breve e la lunghezza media.

In [ ]:
texts = list(df_train['text'])

# analisi delle lunghezze dei tweet espresse in numero di parole
lunghezza_media_tweet = df_train['text'].str.split().apply(len).mean()
lunghezza_massima_tweet = df_train['text'].str.split().apply(len).max()
lunghezza_minima_tweet = df_train['text'].str.split().apply(len).min()

print(f"Max length: {lunghezza_massima_tweet}")
print(f"Min length: {lunghezza_minima_tweet}")
print(f"Mean length: {lunghezza_media_tweet}")

Max length: 84
Min length: 1
Mean length: 15.853546208167025


#**PRE-TRAINING**

Abbiamo deciso di effettuare un pre-training adattivo sul dominio di `BERT-base-cased`, utilizzando `BertForMaskedLM` per la **Masked Language Modeling** (**MLM**) e adattare il modello al linguaggio specifico dei tweet presenti nel nostro dataset.<br>
L'uso della **MLM** consente al modello di apprendere efficacemente anche da dataset di dimensioni ridotte, riducendo il rischio di overfitting. Inoltre, eseguire questa operazione separatamente dall'addestramento sulla classificazione ci permette di migliorare la rappresentazione dei token senza influenzare direttamente la fase di classificazione.

La funzione `maskToken` prende in input: *frase*, *tokenizer* e *max_lenght* (lunghezza massima).

**Tokenizzazione e pre-processing**:<br>
La frase viene tokenizzata, con l'aggiunta dei token speciali `[CLS]` e `[SEP]`.<br>
Viene applicato **padding** fino alla lunghezza massima e, se necessario, **troncamento**.<br>
Il risultato è un dizionario contenente gli ID dei token `input_ids` e la **maschera di attenzione** `attention_mask`, entrambi convertiti in tensori `PyTorch`.

**Mascheratura dei token**:<br>
Mascheratura dei token:
Vengono identificati gli indici dei token validi, escludendo quelli speciali e di padding.
Il 15% dei token validi viene selezionato casualmente per essere trasformato, con un minimo di uno.

I token selezionati vengono suddivisi in tre gruppi:

- 80% mascherati: sostituiti con il token [MASK] (ID 103) e salvati nelle etichette labels.
- 10% sostituiti con token casuali: rimpiazzati con un token scelto casualmente dal vocabolario, escludendo quelli speciali. Anche in questo caso, il valore originale viene salvato in labels.
- 10% lasciati invariati: i token restano inalterati, ma il loro valore originale viene comunque inserito in labels, contribuendo alla funzione di perdita.


**Gestione delle etichette**:<br>
I valori in `labels` che non corrispondono ai token mascherati vengono impostati a `-100`, in modo che **non** contribuiscano alla funzione di perdita durante l'addestramento.

In [ ]:
def maskToken(sentence, tokenizer, max_length=256):
    # Tokenizzazione della frase
    tokens = tokenizer(
        sentence,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors='pt',
    )

    # Estraiamo input_ids e attention_mask
    input_ids = tokens['input_ids'][0]
    attention_mask = tokens['attention_mask'][0]

    # Selezioniamo gli indici dei token validi (cioè non speciali)
    valid_indices = [
        i for i, mask in enumerate(attention_mask)
        if mask == 1 and input_ids[i] not in {tokenizer.cls_token_id, tokenizer.sep_token_id}
    ]

    # Se non ci sono token validi, restituiamo la sequenza invariata e label = -100 per tutti
    if len(valid_indices) == 0:
        return input_ids, attention_mask, torch.full_like(input_ids, -100)

    # Selezioniamo il 15% dei token validi (arrotondando per difetto, ma almeno 1)
    num_to_select = max(1, int(len(valid_indices) * 0.15))
    selected_indices = np.random.choice(valid_indices, size=num_to_select, replace=False)
    selected_indices = list(selected_indices)

    # Cloniamo i token originali per costruire le label
    original_input_ids = input_ids.clone()
    # Inizialmente, le label sono -100
    labels = torch.full_like(input_ids, -100)

    # Se è stato selezionato un solo token, lo mascheriamo
    if num_to_select == 1:
        mask_group = selected_indices
        random_group = []
        unchanged_group = []
    else:
        # Calcolo dei token da processare per ciascuna modalità, arrotondando per difetto
        masked_count = int(0.8 * num_to_select)
        random_count = int(0.1 * num_to_select)
        unchanged_count = int(0.1 * num_to_select)
        # La somma potrebbe essere inferiore a num_to_select: in questo caso i token in più non vengono processati
        total_assigned = masked_count + random_count + unchanged_count

        # Mescoliamo gli indici selezionati per assegnare casualmente le trasformazioni
        np.random.shuffle(selected_indices)
        mask_group = selected_indices[:masked_count]
        random_group = selected_indices[masked_count: masked_count + random_count]
        unchanged_group = selected_indices[masked_count + random_count: masked_count + random_count + unchanged_count]

    # Per i token nel gruppo "mask": sostituiamo con il token [MASK] e impostiamo la label al token originale
    for idx in mask_group:
        input_ids[idx] = tokenizer.mask_token_id
        labels[idx] = original_input_ids[idx]

    # Per i token nel gruppo "random": sostituiamo con un token casuale (non speciale)
    # Costruiamo la lista dei token candidati dalla vocab del tokenizer
    candidate_ids = [
        i for i in range(tokenizer.vocab_size)
        if i not in {tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.mask_token_id}
    ]
    for idx in random_group:
        random_token = np.random.choice(candidate_ids)
        input_ids[idx] = random_token
        labels[idx] = original_input_ids[idx]

    # Per i token nel gruppo "unchanged": lasciamo inalterato il token,
    # ma impostiamo la label al token originale (così viene usato nella loss)
    for idx in unchanged_group:
        labels[idx] = original_input_ids[idx]

    # I token non selezionati restano invariati e con label = -100
    return input_ids, attention_mask, labels

**CONVERSIONE DATAFRAME**

`TweetsDataset` è una sottoclasse di `Dataset`, che ci permette di utilizzarla con un `DataLoader` per fornire i dati al modello durante l'addestramento.<br>
Questa classe converte il `DataFrame` contenente i tweet in un dataset compatibile con `PyTorch`.<br>
Il DataLoader utilizza la funzione collate_fn, che si occupa di:

- Tokenizzare i testi e aggiungere i token speciali ([CLS], [SEP]);
- Applicare la mascheratura dei token secondo la logica della Masked Language Modeling (MLM);
- Restituire un batch di tensori PyTorch

Il batch risultante contiene:

- input_ids → ID dei token mascherati;
- attention_mask → Maschera di attenzione;
- labels → Etichette per l'addestramento con MLM, con -100 per i token non mascherati.



In [ ]:
class TweetsDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data.iloc[index]['text']

def collate_fn(batch, tokenizer, max_length=256):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    for text in batch:
        input_ids, attention_mask, labels = maskToken(text, tokenizer, max_length)
        input_ids_list.append(input_ids.squeeze(0))
        attention_mask_list.append(attention_mask.squeeze(0))
        labels_list.append(labels.squeeze(0))


    return {
        "input_ids": torch.stack(input_ids_list),
        "attention_mask": torch.stack(attention_mask_list),
        "labels": torch.stack(labels_list)
    }

**IPERPARAMETRI DEL FINE-TUNING**

In [ ]:
hyperparameters = {

    'epochs': 5,

    'batch_size': 64,

    'dropout': 0.3,

    'learning_rate': 1e-5

}

**CREAZIONE DEL DATASET E ISTANZIAZIONE DEL DATALOADER**

Il `DataLoader` di `PyTorch` permette di caricare e gestire i dati in `batch` durante l'addestramento del modello:

*   Crea mini-batch di dati a partire dal `dataset` che gli viene fornito
*   Shuffla i dati, aiutando a migliorare la generalizzazione del modello

In [ ]:
import os
print(os.cpu_count())

12


In [ ]:
from functools import partial

tweets_dataset = TweetsDataset(df_train)

dataloader = DataLoader(
    dataset=tweets_dataset,
    batch_size=hyperparameters['batch_size'],
    shuffle=True,
    num_workers=6,  # Parallelizza il lavoro di pre-elaborazione
    collate_fn=partial(collate_fn, tokenizer=tokenizer, max_length=256)
)

**PRE-TRAINING CON MLM**

Abbiamo deciso di non addestrare tutti i layer di `BERT-base-cased`, poiché i primi strato catturano principalmente feature linguistiche generali di basso livello. Se avessimo addestrato l'intero modello, questi layer avrebbero dovuto riapprendere caratteristiche già consolidate, aumentando il rischio di overfitting e rendendo l'addestramento meno efficiente.<br>
Un altro fattore cruciale nella nostra scelta è stato il costo computazionale: l'addestramento di `BERT-base-cased` è già computazionalmente intensivo, e congelare alcuni layer riduce il numero di gradienti da calcolare.

**Adam** (**Adaptive Moment Estimation**) è un ottimizzatore che combina **momentum** (accellera la convergenza e stabilizza l'aggiornamento dei pesi) e **adaptive learning rate** (bilancia l'aggiornamento di pesi con scale diverse) per aggiornare i pesi in modo più stabile.<br>
Abbiamo scelto la variante **AdamW**, che applica la *weight decay* (penalizza i pesi grandi) come una penalizzazione separata dai pesi, senza interferire direttamente con i gradienti. Questo aiuta a ridurre l'overfitting, specialmente su dataset di dimensioni limitate.

**Gestione del learning rate**:<br>
`num_training_step` indica il numero totale di iterazioni di aggiornamento dei pesi.<br>
`lr_scheduler` definisce la strategia di variazione del *learning rate* durante l'addestramento. Abbiamo utilizzato uno scheduler lineare `linear` in cui il learning rate decresce linearmente fino a zero. `num_warmup_steps` significa che nei primi 10% degli step, il *learning rate* aumenta prima di iniziare a decrescere, stabilizzando l'addestramento nelle fasi iniziali.

**Gestione della memoria GPU**<br>
Per evitare di superare i limiti della memoria GPU, utilizziamo `torch.cuda.empty_cache()`, che libera la memoria inutilizzata, ottimizzando l'uso delle risorse hardware.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# impostiamo il modello in modalità di addestramento, da ora aggiornerà i pesi
model_lm.train()

# congeliamo i primi 8 layer, evitando di congelarli tutti (compresi i layer di embedding)
for layer in model_lm.bert.encoder.layer[:-4]:
    for param in layer.parameters():
        param.requires_grad = False

#spostiamo il modello su GPU
model_lm.to(device)

#definiamo dell'ottimizzatore
optimizer = AdamW(model_lm.parameters(), lr=hyperparameters['learning_rate'], weight_decay=0.01)

#scheduler per il decay del learning rate
num_training_steps = len(dataloader) * hyperparameters['epochs']
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=int(0.1 * num_training_steps), num_training_steps=num_training_steps
)

lrs = []
#loop di addestramento
progress_bar_epochs = tqdm(range(hyperparameters['epochs']), desc="Training Progress")
for epoch in progress_bar_epochs:
    torch.cuda.empty_cache()
    total_loss = 0
    progress_bar_batches = tqdm(dataloader, desc=f"Epoch {epoch+1}", leave=False)
    for batch in progress_bar_batches:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model_lm(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        lrs.append(optimizer.param_groups[0]['lr'])
        total_loss += loss.item()

        # ggiornamento della barra di progresso per i batch
        progress_bar_batches.set_postfix(loss=loss.item())

        #Pulizia memoria GPU
        outputs = outputs.logits.detach().cpu()
        input_ids = input_ids.detach().cpu()
        attention_mask = attention_mask.detach().cpu()
        labels = labels.detach().cpu()
    avg_loss = total_loss / len(dataloader)
    progress_bar_epochs.set_postfix(avg_loss=avg_loss)


#Salviamo il modello nel formato di Hugging Face
model_save_path = "/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/bert_mlm2"
tokenizer_save_path = "/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/tokenizer2"
model_lm.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

cuda


Training Progress: 100%|██████████| 5/5 [20:06<00:00, 241.36s/it, avg_loss=3.33]


('/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/tokenizer2/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/tokenizer2/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/tokenizer2/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/tokenizer2/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/tokenizer2/tokenizer.json')

**CARICAMENTO DEL TOKENIZER ADDESTRATO**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/tokenizer2")
# Verifica la dimensione del vocabolario dopo l'aggiunta dei nuovi token
print("Dimensione aggiornata del vocabolario del tokenizer:", len(tokenizer))

Dimensione aggiornata del vocabolario del tokenizer: 29179


**CARICAMENTO DEL MODELLO PRE-TRAINED**

In [ ]:
model_lm = AutoModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/bert_mlm2")

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/pretrained_models/bert_mlm2 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Verifica la dimensione della matrice di embedding nel modello
print("Dimensione della matrice di embedding del modello:", model_lm.get_input_embeddings().weight.size(0))

Dimensione della matrice di embedding del modello: 29179


In [ ]:
print(model_lm)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29179, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

#**FINE-TUNING SULLA CLASSIFICAZIONE**



Riassegnamo a *df_train* la variabile *df_updated*, su cui avevamo precedentemente caricato il dataset dopo aver effettuato la **Data Augmentation**.<br>
In questo modo ci assicuriamo di effettuare il **pre-training** sul dataset originale ed il **fine-tuning** sul dataset aumentato.<br>
Effettuiamo lo *shuffle* poiché i campioni "sintetici" erano stati aggiungi in blocchi per ognuna delle classi. Evitiamo di avere troppi tweet consecutivi con la stessa classe.

In [ ]:
df_train = df_updated
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_train

,text,labels
0,ditto!! Such an amazing atmosphere! #PhilippPl...,1
1,unless your concern is people figuring out who...,0
2,incredibly terrible about life is i ' m 23 yea...,3
3,laughing out loud I have reminders for my ex-g...,1
4,"optimism is atomic number 2 ' ll lose, that ' ...",2
...,...,...
4768,Kids often cry when they can't seem to find th...,3
4769,I've got #teampaella presents on their way for...,3
4770,Ill say it again. If I was a Black man Id be a...,3
4771,"The point of life, and to be an optimist, is t...",2


##**DEFINIZIONI INIZIALI**

**CONTROLLO DEL TIPO DI DATI**


In [ ]:
print(df_train['labels'].dtype)  # Controlla il tipo di dati
print(type(df_train['labels'][0]))          # Controlla il tipo del primo elemento

int64
<class 'numpy.int64'>


**CONVERSIONE DELLE LABELS IN INTERI**

In [ ]:
df_train['labels'] = df_train['labels'].astype(int)
y_train = df_train['labels'].tolist()  # Assicuriamoci che y_train sia una lista di interi
df_val['labels'] = df_val['labels'].astype(int)
y_val = df_val['labels'].tolist()      # Assicuriamoci che y_train sia una lista di interi

**CONTROLLO DI AVVENUTA CONVERSIONE**

In [ ]:
print(df_train['labels'].dtype)  # Controlla il tipo di dati
print(type(df_train['labels'][0]))          # Controlla il tipo del primo elemento

int64
<class 'numpy.int64'>


**DIZIONARIO DELLE ETICHETTE**

In [ ]:
labels_dict = {
    'anger': 0,
    'joy': 1,
    'optimism': 2,
    'sadness': 3
}

Separiamo i dati dalle etichette per ciascun dataset (per il dataset di test abbiamo solamente i dati)

In [ ]:
X_train = df_train['text']
y_train = df_train['labels']
X_val = df_val['text']
y_val = df_val['labels']
X_test = df_test['text']

##**CREAZIONE DEI DATASET**

**METODO PER CREARE I DATASET CON ETICHETTE (se disponibili)**

`Dataset` accetta in input:
*   Una lista di frasi `x`
*   Una lista di etichette opzionale `y`
*   Un tokenizer
*   La lunghezza massima dei token `max_length`

Il metodo `__getitem__` estrae il testo corrispondente all'indice `idx` e lo tokenizza. La tokenizzazione include:
*   Troncamento se la lunghezza supera `max_length`
*   Padding se la lunghezza è inferiore a `max_length`
*   Aggiunta dei token speciali `[CLS]` e `[SEP]`

Viene generato il dizionario `batch_texts` contenente `input_ids` con gli ID dei token generati dal tokenizer e `attention_mask` che indica con `1` i token effettivi e con `0` i token di padding.

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, x, y=None, tokenizer=None, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = list(x)
        self.labels = [torch.tensor(label) for label in y] if y is not None else None

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenizza i testi usando il tokenizer di BERT
        tokens = self.tokenizer(
            self.texts[idx],
            add_special_tokens=True,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors='pt'
        )
        batch_texts = {
            'input_ids': tokens['input_ids'].squeeze(0),
            'attention_mask': tokens['attention_mask'].squeeze(0)
        }

        # Se le etichette sono disponibili vengono restituite
        if self.labels is not None:
            batch_labels = self.labels[idx]
            return batch_texts, batch_labels
        else:
            return batch_texts



**CREAZIONE DEI DATASET**

`train_dataset` con testi `X_train` e labels `y_train`<br>
`val_dataset` con testi `x_val` e labels `y_val`<br>
`test_dataset` con testi `x_test`

In [ ]:
# Creazione dei dataset per train e validation (con etichette)
train_dataset = Dataset(X_train, y_train, tokenizer)
val_dataset = Dataset(X_val, y_val, tokenizer)
# Creazione del dataset per test (senza etichette)
test_dataset = Dataset(X_test, tokenizer=tokenizer)

**PRIMO CAMPIONE DEL DATASET DI TRAINING**

In [ ]:
train_dataset[0]

({'input_ids': tensor([  101,  4267,  8849,   106,   106,  5723,  1126,  6929,  6814,   106,
            108, 22765,  2101, 18929,   108, 18456, 20547,  1116,   108, 18151,
            108,  1498,  2036, 14850,  1116,   108, 18456,   102,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,    

##**CLASSIFICATORE**

Fin dall'inizio abbiamo deciso usare la rete ricorrente **LSTM** per rielaborare l'intero output del modello **BERT-base-cased**. Abbiamo comunque provato delle alternative per valutarne le prestazioni:

**1. BERT con Feed-Forward**<br> L'outoyt du BERT viene passato a una rete *feed-forward*. Per la classificazione viene utilizzato il token `[CLS]`, ottenuto dall'ultimo *hidden state* di BERT (`last_hidden_state[:, 0, :]`).<br>
Questa soluzione ha mostrato prestazioni inferiori rispetto alle altre. Il motivo principale è che il token `[CLS]` potrebbe non catturare sufficientemente le relazioni tra parole distanti, specialmente nei tweet.
<br><br>
**2. BERTForMaskedLM con Feed-Forward**<br> A differenza della prima versione, **BERTForMaskedLM** viene utilizzato per ottenere tutti gli *hidden states* del modello (`output_hidden_states=True`). L'ultimo (`hidden_state[-1]`) viene usato per estrarre il token `[CLS]`, che viene poi passato a una rete *feed-forward*. Le prestazioni sono risultate migliori rispetto alla opzione 1, ma ancora una volta inferiori al nostro modello.
<br><br>
**3. BERT con LSTM - scelta iniziale (confermata)**<br> L'introduzione della **LSTM** è stata motivata dalla volontà di preservare le relazioni tra parole distanti e migliorare la rappresentazione del testo. **BERT**, infatti, utilizza *self-attention* per modellare il contesto, ma non mantiene una memoria esplicita della sequenza, il che può risultare limitante per la classificazione di tweet, che spesso hanno strutture irregolari.<br>
BERT estrae gli *hidden states* della sequenza in ingresso, fornendo rappresentazioni contestuali avanzate.<br>
La **BiLSTM** elabora questi *hidden states* per catturare le relazioni sequenziali tra parole.<br>
L'ultimo stato nascosto della **LSTM** viene passato a un classificatore, che predice l'etichetta finale del tweet.

I pro del nostro approccio:


*   La **LSTM** mantiene una traccia esplicita della sequenza, cosa che l'attenzione di **BERT** da sola non può fare
*   A differenza delle altre soluzioni, la **LSTM** sfrutta tutte le rappresentazioni dei token per generare una previsione più accurata
*   Analizzare il testo sia da sinistra a destra che viceversa migliora la capacità del modello di comprendere la struttura del tweet, dove il significato di una parola può dipendere sia da ciò che la precede sia da ciò che la segue
*   Congiungendo la capacità di **BERT** di apprendere rappresentazioni contestuali con la capacità della **LSTM** di preservare la sequenza, il modello è risultato più stabile ed efficace.

In [ ]:
import torch
import torch.nn as nn


class BERTBiLSTMClassifier(nn.Module):
    def __init__(self, model, labels, lstm_layers, lstm_hdim, bilstm, dropout):
        super(BERTBiLSTMClassifier, self).__init__()
        # BERT pre-trained model
        self.bert = model

        # LSTM layer
        self.lstm = nn.LSTM(self.bert.config.hidden_size, lstm_hdim, lstm_layers,
                            bidirectional=bilstm, batch_first=True)

        # Output dimension of LSTM
        lstm_output = lstm_hdim * 2 if bilstm else lstm_hdim
        self.bilstm = bilstm

        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(lstm_output, lstm_output),
            nn.BatchNorm1d(lstm_output),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(lstm_output, labels)
        )

    def forward(self, input_ids, attention_mask):
        # Output di BERT
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        # Usiamo l'ultimo hidden state
        lstm_input = bert_output.last_hidden_state  # Shape: (batch_size, seq_len, hidden_size)


        lstm_input = lstm_input[:, 1:-1, :]  # Rimuoviamo [CLS] e [SEP]
        attention_mask = attention_mask[:, 1:-1]  # Aggiorniamo anche la mask

        # LSTM processing
        _output, (hn, _cn) = self.lstm(lstm_input)

        # Estrarre gli stati finali
        if not self.bilstm:
            lstm_output = hn[-1, :, :]  # Ultimo stato nascosto
        else:
            lstm_output = torch.cat((hn[-2, :, :], hn[-1, :, :]), dim=1)  # Ultimi due stati per BiLSTM

        final = self.classifier(lstm_output)
        return final


##**ADDESTRAMENTO E CLASSIFICAZIONE**

**EARLY STOPPING**

L'**early stopping** monitora la *loss* di validazione (`validation_loss`) durante il training e interrompe l'addestramento se non si osservano miglioramenti per un numero consecutivo di epoche definito da `patience`.<br>
Previene l'overfitting, fermando il training prima che il modello inizi a memorizzare i dati di training invece di generalizzarli.

Questo approcio ottimizza il trade-off tra accuratezza e generalizzazione, evitando di sprecare risorse computazionali su epoche inutili.

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):

        self.patience = patience
        self.min_delta = min_delta              # valore minimo di decrescita della loss di validazione all'epoca corrente
                                                # per asserire che c'è un miglioramenti della loss
        self.counter = 0                        # contatore delle epoche di pazienza
        self.early_stop = False                 # flag di early stop
        self.min_validation_loss = torch.inf    # valore corrente ottimo della loss di validazione

    def __call__(self, validation_loss):
        # chiamata in forma funzionale dell'oggetto di classe EarlySopping

        if (validation_loss + self.min_delta) >= self.min_validation_loss:  # la loss di validazione non decresce
            self.counter += 1                                               # incrementiamo il contatore delle epoche di pazienza
            if self.counter >= self.patience:
                self.early_stop = True
                print("Early stop!")
        else:                                               # c'è un miglioramento della loss:
            self.min_validation_loss = validation_loss      # consideriamo la loss corrente
                                                            # come nuova loss ottimale
            self.counter = 0                                # e azzeriamo il contatore di pazienza

**CICLO DI ADDESTRAMENTO DI UN'EPOCA**

Viene impostato il modello in modalità *train* tramite `model.train()`.<br>
Itera sui batch del `dataloader`, che contiene gli input tokenizzati (`input_ids`, `attention_mask`) e le etichette (`batch_labels`).<br>
Esegue il passo di `forward`, passando i dati al modello.<br>
Calcola la *loss* con la funzione di perdita `loss(output, batch_labels)`.<br>
Esegue la *backpropagation* (`.backward()` e aggiorna i pesi (`.step()`).<br>
Usa la `LogSoftmax` per normalizzare le probabilità, confrontando la classe predetta con quella reale.<br>
Vengono tolti i tensori dalla GPU per evitare *memory leak* (`detach().cpu()`).<br>
Restituisce la *loss media* come perdita dell'epoca divisa per la lunghezza del dataloader (`epoch_loss / len(dataloader)`) e l'*accuracy* su tutti i *batch* (`epoch_acc`).

In [ ]:
from sklearn.metrics import f1_score

def train_loop(model, dataloader, loss, optimizer, device):
    model.train()

    epoch_acc = 0
    epoch_loss = 0

    for batch_texts, batch_labels in tqdm(dataloader, desc='training set'):


        optimizer.zero_grad()

        #Accediamo direttamente agli input tokenizzati
        input_id_texts = batch_texts['input_ids'].to(device)
        mask_texts = batch_texts['attention_mask'].to(device)
        batch_labels = batch_labels.to(device)

        output = model(input_id_texts, mask_texts)

        #Calcoliamo la loss
        batch_loss = loss(output, batch_labels)
        batch_loss.backward()
        optimizer.step()

        epoch_loss += batch_loss.item()

        #Calcoliamo l'accuracy
        softmax = nn.LogSoftmax(dim=1)
        epoch_acc += (softmax(output).argmax(dim=1) == batch_labels).sum().item()

        batch_labels = batch_labels.detach().cpu()
        input_id_texts = input_id_texts.detach().cpu()
        mask_texts = mask_texts.detach().cpu()
        output = output.detach().cpu()

    #Loss media e accuratezza totale
    return epoch_loss / len(dataloader), epoch_acc

**UN CICLO DI CLASSIFICAZIONE DEL SET DI VALIDAZIONE / TEST**

Si imposta il modello in modalità *eval* (`model.eval()`) per disattivare *dropout* e *batch norm* (se presenti) per stabilizzare la valutazione.<br>
Si disattiva il calcolo dei *gradienti* per risparmiare memoria e velocizzare l'inferenza, dato che non serve aggiornare ulteriormente i pesi.<br>
Itera sui batch del *dataloader* di *test* o *validation*, passa gli input al modello (`input_ids` e `attention_mask`), calocla la *loss* per monitorare l'errore, calcola l'accuratezza normalizzando le probabilità con `LogSoftmax` e confrontando le predizioni (`output`) con le etichette reali (`batch_labels`).

In [ ]:
def test_loop(model, dataloader, loss, device):
    model.eval()

    epoch_acc = 0
    epoch_loss = 0

    with torch.no_grad():
        for batch_texts, batch_labels in tqdm(dataloader, desc='dev set'):
            #Accediamo direttamente agli input tokenizzati
            input_id_texts = batch_texts['input_ids'].to(device)
            mask_texts = batch_texts['attention_mask'].to(device)
            batch_labels = batch_labels.to(device)

            output = model(input_id_texts, mask_texts)

            #Calcoliamo la loss
            batch_loss = loss(output, batch_labels)
            epoch_loss += batch_loss.item()

            # Calcola l'accuracy
            softmax = nn.LogSoftmax(dim=1)
            epoch_acc += (softmax(output).argmax(dim=1) == batch_labels).sum().item()

    #Loss media e accuratezza totale
    return epoch_loss / len(dataloader), epoch_acc



**LOOP COMPLETO DI ADDESTRAMENTO E VALIDAZIONE**

Vengono inizialmente creati i *dataloader* dei dataset di *train* e *validation*, per caricarli in batch, migliorando l'efficienza del training.<br>
Esegue il training per un'epoca intera con la funzione `train_loop()`, salva la *loss* e l'*accuracy* per il monitoraggio (`train_loss`, `train_acc`).<br>
Se l'*early stopping* è attivato, calcola la *loss* e l'*accuracy* sul set di *validation* usando `test_loop()`. Viene anche controllato se la *loss* di validazione smette di migliorare. Se il *training* si è stabilizzato per `patience` epoche, lo interrompe automaticamente.

Se lo `scheduler` è presente, aggiorna il *learning rate* o altri iperparametri se è passato uno `scheduler`.

Abbiamo scelto di eseguire il metodo `test_loop` su `val_dataloader` ad ogni epoca per poter valutare le prestazioni del modello anche in epoche intermedie.

In [ ]:
def train_test(model, epochs, optimizer, device, train_data, test_data,
               batch_size, train_loss_fn,
               test_loss_fn=None,         # Non necessariamente train e test loss devono differire
               early_stopping=None,       # Posso addestrare senza early stopping
               val_data=None,             # E in questo caso non c'è validation set
               scheduler=None):           # Possibile scheduler per monitorare l'andamento di un iperparametro,
                                          # tipicamente il learning rate

    # Creazione dei DataLoader
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, drop_last=True)

    # Check sulle funzioni di loss
    if test_loss_fn is None:
        test_loss_fn = train_loss_fn

    # Liste dei valori di loss e accuracy epoca per epoca
    train_loss = []
    validation_loss = []
    test_loss = []

    train_acc = []
    validation_acc = []
    test_acc = []

    # Ciclo di addestramento con early stopping
    for epoch in tqdm(range(1, epochs + 1)):

        # Train
        epoch_train_loss, epoch_train_acc = train_loop(model, train_dataloader, train_loss_fn, optimizer, device)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_acc / len(train_data))


        # Validation
        if early_stopping != None:
            epoch_validate_loss, epoch_validate_acc = test_loop(model, val_dataloader, test_loss_fn, device)
            validation_loss.append(epoch_validate_loss)
            validation_acc.append(epoch_validate_acc / len(val_data))

        # Logging
        val_loss_str = f'Validation loss: {epoch_validate_loss:6.4f} ' if early_stopping != None else ' '
        val_acc_str = f'Validation accuracy: {(epoch_validate_acc / len(val_data)):6.4f} ' if early_stopping != None else ' '
        print(f"\nTrain loss: {epoch_train_loss:6.4f} {val_loss_str}")
        print(f"Train accuracy: {(epoch_train_acc / len(train_data)):6.4f} {val_acc_str}")

        # Early stopping
        if early_stopping != None:
            early_stopping(epoch_validate_loss)
            if early_stopping.early_stop:
                break

        # Scheduler update
        if scheduler != None:
            scheduler.step()

    return train_loss, validation_loss, test_loss, train_acc, validation_acc, test_acc



**PROVA DI VARI IPERPARAMETRI**

Il metodo `hyperparameter_search` usa la classe `BERTBiLSTMClassifier` a cui passiamo il nostro modello.<br>
L'utilità è quella di provare vari iperparametri per la classificazione, definiti nel vocabolario `hyperparameters_grid`.

In [ ]:
# Acquisiamo il device su cui effettueremo il training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def hyperparameter_search(model_lm, train_data, val_data, test_data, hyperparameter_grid, device):
    best_hyperparameters = {}

    # Ciclo attraverso tutte le combinazioni di iperparametri
    for params in hyperparameter_grid:
        print(f"Training with hyperparameters: {params}")

        model = BERTBiLSTMClassifier(model_lm, len(labels_dict), lstm_layers=params['layers'],
                              lstm_hdim=params['h_dim'], bilstm=params['bilstm'], dropout=params['dropout']).to(device)
        print(model)

        optimizer = Adam(model.parameters(), lr=params["learning_rate"], weight_decay=0.021943967420409143)
        criterion = nn.CrossEntropyLoss()

        early_stopping = EarlyStopping(patience=params['patience'], min_delta=params['min_delta'])

        # Eseguiamo il training con il set di iperparametri corrente
        train_loss, validation_loss,test_loss, train_acc, validation_acc, test_acc = train_test(
            model,
            params["epochs"],
            optimizer,
            device,
            train_data,
            test_data,
            params["batch_size"],
            criterion,
            criterion,
            early_stopping,
            val_data)

    return model

Using cuda device


**IPERPARAMETRI PER L'ADDESTRAMENTO ALLA CLASSIFICAZIONE**

Lo scopo della lista di dizionari *hyperparameter_grid* era quello di effettuare vari addestramenti per determinare la combinazione migliore di iperparametri. Per ottimizzare tale fase, abbiamo scelto di utilizzare la libreria `Optuna`, una libreria di ottimizzazione bayesiana che consente di esplorare automaticamente lo spazio degli iperparametri (cella successiva).<br>
La combinazione attualmente presente in `hyperparameter_grid` è risultata essere quella che minimizza la loss sul validation set.

In [ ]:
hyperparameter_grid = [
    {"epochs": 15, "learning_rate": 3.092111511823065e-06, "batch_size":64, "dropout": 0.14958798350199062, "h_dim": 768, "patience": 3, "min_delta": 0.01, "layers": 1, "bilstm": False},
]

# Eseguiamo la ricerca degli iperparametri
model_new = hyperparameter_search(model_lm, train_dataset, val_dataset, test_dataset, hyperparameter_grid, device)

Training with hyperparameters: {'epochs': 15, 'learning_rate': 3.092111511823065e-06, 'batch_size': 64, 'dropout': 0.14958798350199062, 'h_dim': 768, 'patience': 3, 'min_delta': 0.01, 'layers': 1, 'bilstm': False}
BERTBiLSTMClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29179, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Drop

training set: 100%|██████████| 74/74 [00:44<00:00,  1.65it/s]

  7%|▋         | 1/15 [00:46<10:44, 46.01s/it]


Train loss: 1.3813 Validation loss: 1.2631 
Train accuracy: 0.3256 Validation accuracy: 0.3797 



training set: 100%|██████████| 74/74 [00:43<00:00,  1.69it/s]

 13%|█▎        | 2/15 [01:30<09:48, 45.26s/it]


Train loss: 1.2314 Validation loss: 1.0787 
Train accuracy: 0.4511 Validation accuracy: 0.5080 


...


Train loss: 0.4604 Validation loss: 0.5939 
Train accuracy: 0.8320 Validation accuracy: 0.6711 



training set: 100%|██████████| 74/74 [00:43<00:00,  1.69it/s]

 80%|████████  | 12/15 [08:58<02:14, 44.81s/it]


Train loss: 0.4384 Validation loss: 0.5915 
Train accuracy: 0.8443 Validation accuracy: 0.6818 



training set: 100%|██████████| 74/74 [00:43<00:00,  1.69it/s]

 80%|████████  | 12/15 [09:43<02:25, 48.62s/it]


Train loss: 0.4091 Validation loss: 0.6005 
Train accuracy: 0.8521 Validation accuracy: 0.6791 
Early stop!


In [ ]:
print(model_new)

BERTBiLSTMClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29179, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
!pip install optuna
import optuna
import gc
from torch.optim import Adam
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def objective(trial):

    # Impostiamo valori fissi
    epochs = 8  # Numero fisso di epoche
    patience = 2  # Numero fisso di epoche di pazienza per l'early stopping
    min_delta = 0.01  # Soglia fissa per il miglioramento minimo

    # Definizione degli iperparametri da ottimizzare
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-2, 1e-1)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    dropout = trial.suggest_uniform("dropout", 0.1, 0.5)
    lstm_hdim = trial.suggest_int("lstm_hdim", 768, 1024, step=128)
    lstm_layers = trial.suggest_int("lstm_layers", 1, 2)
    bilstm = trial.suggest_categorical("bilstm", [True, False])

    # Stampiamo degli iperparametri provati in questo trial
    print(f"Trial {trial.number}:")
    print(f"  learning_rate: {learning_rate}")
    print(f"  weight_decay: {weight_decay}")
    print(f"  batch_size: {batch_size}")
    print(f"  dropout: {dropout}")
    print(f"  lstm_hdim: {lstm_hdim}")
    print(f"  lstm_layers: {lstm_layers}")
    print(f"  bilstm: {bilstm}")
    print("-" * 40)


    # Modello
    model = BERTBiLSTMClassifier(
        model=model_lm,
        labels=len(labels_dict),
        lstm_layers=lstm_layers,
        lstm_hdim=lstm_hdim,
        bilstm=bilstm,
        dropout=dropout,
    ).to(device)

    optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)

    # Training del modello
    train_loss, validation_loss, _, train_acc, validation_acc, _ = train_test(
        model,
        epochs,
        optimizer,
        device,
        train_data=train_dataset,
        test_data=test_dataset,
        batch_size=batch_size,
        train_loss_fn=criterion,
        test_loss_fn=criterion,
        early_stopping=early_stopping,
        val_data=val_dataset,
    )

    # Pulizia della memoria dopo ogni prova
    del model
    del optimizer
    torch.cuda.empty_cache()  # Svuota la memoria GPU
    gc.collect()  # Svuota la RAM

    # Restituiamo la loss di validazione come metrica da minimizzare
    return validation_loss[-1] if validation_loss else float('inf')

# Creazione dello studio
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20, n_jobs=1, gc_after_trial=True)


print("Best trial:")
print(study.best_trial.params)


[I 2025-02-12 21:27:28,674] A new study created in memory with name: no-name-19c0a15a-fa9e-4a78-9571-b374a9c58f11


Using cuda device
Trial 0:
  learning_rate: 3.8070468229347063e-06
  weight_decay: 0.03979655363873826
  batch_size: 64
  dropout: 0.3881224682805564
  lstm_hdim: 896
  lstm_layers: 1
  bilstm: True
----------------------------------------


<ipython-input-42-96d4061d4e79>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
<ipython-input-42-96d4061d4e79>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-2, 1e-1)
<ipython-input-42-96d4061d4e79>:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform("dropout", 0.1, 0.5)
training set: 100%|██████████| 74/74 [00:45<00:00,  1.63it/s]

 12%|█▎        | 1/8 [00:46<05:26, 


Train loss: 1.4308 Validation loss: 1.2631 
Train accuracy: 0.3195 Validation accuracy: 0.3824 



training set: 100%|██████████| 74/74 [00:44<00:00,  1.67it/s]

 25%|██▌       | 2/8 [01:32<04:35, 45.93s/it]

...


Train loss: 0.1010 Validation loss: 0.9993 
Train accuracy: 0.9834 Validation accuracy: 0.6898 



training set: 100%|██████████| 298/298 [00:58<00:00,  5.13it/s]

 25%|██▌       | 2/8 [02:58<08:56, 89.49s/it]



Train loss: 0.0688 Validation loss: 1.0606 
Train accuracy: 0.9858 Validation accuracy: 0.7059 
Early stop!


[I 2025-02-12 22:32:26,965] Trial 19 finished with value: 1.060552170743113 and parameters: {'learning_rate': 1.5068704222336597e-06, 'weight_decay': 0.010032045865953592, 'batch_size': 16, 'dropout': 0.17702880505007257, 'lstm_hdim': 768, 'lstm_layers': 2, 'bilstm': True}. Best is trial 3 with value: 0.5610706388950348.


Best trial:
{'learning_rate': 3.092111511823065e-06, 'weight_decay': 0.021943967420409143, 'batch_size': 64, 'dropout': 0.14958798350199062, 'lstm_hdim': 768, 'lstm_layers': 1, 'bilstm': False}


In [ ]:
print("Best validation loss:", study.best_trial.value)

Best validation loss: 0.5610706388950348


**SALVATAGGIO DEL MODELLO**

In [ ]:
import os

def save_model_only(net, path):
    torch.save({
        'model_state_dict': net.state_dict(),
    }, path)

model_path = "/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/modellofinaleoptuna15"

# Crea la cartella se non esiste
os.makedirs(os.path.dirname(model_path), exist_ok=True)

save_model_only(model_new, model_path)

#**CARICAMENTO E UTILIZZO DEL MODELLO**

##**CARICAMENTO DEL MODELLO**

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/modellofinaleoptuna15"
params = {"epochs": 15, "learning_rate": 3.092111511823065e-06, "batch_size":64, "dropout": 0.14958798350199062, "h_dim": 768, "patience": 3, "min_delta": 0.01, "layers": 1, "bilstm": False}
model_lm = BertModel.from_pretrained("bert-base-cased")
model_lm.resize_token_embeddings(len(tokenizer))
def load_model_only(path):
    checkpoint = torch.load(path)
    model = BERTBiLSTMClassifier(model_lm, len(labels_dict), lstm_layers=params['layers'],
                              lstm_hdim=params['h_dim'], bilstm=params['bilstm'], dropout=params['dropout'])
    model.load_state_dict(checkpoint['model_state_dict'])
    return model
mod = load_model_only(model_path)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<ipython-input-36-8e9f601c6273>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


In [ ]:
mod.to(device)

BERTBiLSTMClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29179, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

##**PREDIZIONE DELLE ETICHETTE**

Il metodo `evaluate` permette di effettuare la classificazione su un dataset di test o validazione, generando un file `.csv` con le predizioni del modello.

Si chiama `model.eval()` per disattivare il *dropout* e la *batch normalization*, garantendo una valutazione più stabile.<br>
Si usa `torch.no_grad()` per risparmiare memoria ed evitare calcoli inutili (i pesi non devono essere aggiornati).<br>
Se si vuole classificare il **test set**, il metodo utilizza `test_dataloader`, ma se si vuole classificare il **validation set**, bisogna:
*   Decommentare la riga `val_dataloader`
*   Cambiare `test_dataloader` con `val_dataloader` nel ciclo `for`
*   Aggiungere `_` nel ciclo `for`, poiché il **validation set** include anche le etichette reali

Si itera sui batch del *dataloader* e vengono estratti `input_ids` e `attention_mask`, che vengono passati al modello.<br>
L'output del modello viene trasformato in probabilità con la **Softmax** (`nn.LogSoftmax(dim=1)`) e si seleziona la classe con probabilità massima (`argmax(dim=1)`).

Si salvano le predizioni in un file `.csv` che ha come intestazione "label", e ogni riga contiene l'etichetta predetta per un tweet.<br>
Per garantire una corretta gestione della memoria, dopo ogni batch si rilascia la GPU con `.detach().cpu()`.

In [ ]:
def evaluate(model, device, pred_path):
    model.eval()

    #val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1)


    with torch.no_grad():
        with open(pred_path, 'w') as f:
            f.write('label\n')  # Header per il file CSV

        for batch_texts in tqdm(test_dataloader, desc='evaluation'):
            # Accediamo direttamente agli input tokenizzati
            input_id_texts = batch_texts['input_ids'].to(device)
            mask_texts = batch_texts['attention_mask'].to(device)

            # Ottieniamo l'output del modello
            output = model(input_id_texts, mask_texts)

            # Calcola le predizioni
            softmax = nn.LogSoftmax(dim=1)
            predictions = softmax(output).argmax(dim=1)

            #batch_labels = batch_labels.detach().cpu()
            input_id_texts = input_id_texts.detach().cpu()
            mask_texts = mask_texts.detach().cpu()
            output = output.detach().cpu()

            # Scriviamo le predizioni nel file
            with open(pred_path, 'a') as f:
                for prediction in predictions:
                    f.write(f"{prediction.item()}\n")  # Usiamo item() per ottenere un valore scalare


**AVVIO DELLA PREDIZIONE SUL TEST**

Si carica il modello sulla GPU se è disponibile.<br>
Si stabilisce il path su cui salvare il file `.csv` contenente le predizioni.<br>
Si avvia il metodo `evaluate` a cui passiamo il modello caricato, il dispositivo su cui deve runnare e il percorso su cui salvare le predizioni (`mod`, `device`, `predictions_path`).

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predictions_path = '/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/predictions/predictionsprovefinali/predictionstest15.csv'
evaluate(model_new, device, predictions_path)

evaluation: 100%|██████████| 1421/1421 [00:22<00:00, 63.90it/s]


##**F1-SCORE SUL DATASET DI VALIDATION**

Si prendono le vere etichette `y_true` dal DataFrame `df_val['labels']` e si confrontano con le `y_pred` salvate sul file `predictionsval.csv`.

In [ ]:
from sklearn.metrics import f1_score

y_true = df_val['labels'].tolist()
y_pred = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Per un pugno di cfu/emotion/predictions/predictionsprovefinali/predictionsval2.csv')['label'].tolist()

f1_w = f1_score(y_true, y_pred, average='weighted')  # 'weighted' gestisce il caso di classi squilibrateprint(f'F1-score: {f1:.4f}')

f1_macro = f1_score(y_true, y_pred, average='macro') # calcola l'F1-score per ogni classe e fa la media aritmetica (tratta tutte le classi allo stesso modo)

f1_micro = f1_score(y_true, y_pred, average='micro') # calcola la metrica considerando globalmente i TP, FP e FN (utile per multilabel)

print(f'F1-score_weighted: {f1_w:.4f}')
print(f'F1-score_macro: {f1_macro:4f}')
print(f'F1-score_micro: {f1_micro:4f}')

F1-score_weighted: 0.7814
F1-score_macro: 0.721570
F1-score_micro: 0.775401
